# Statistical forecasting : energy demand
---

### Time series definition

Time series = Signal + Noise

**Signal** : forecasts extrapolate signal portion of model
  - Trend/Cycle
  - Seasonal

**Noise** : confidence intervals account for uncertainty
  - Error/Residuals/Remainder/Irregularities

## Data construction

### Settings

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

#### libraries

In [ ]:
import cv2
import numpy as np, warnings, itertools
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import scipy.stats as st
import statsmodels.api as sm
import statsmodels.formula.api as smf

from math import sqrt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import *
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.stats import kstest, norm, shapiro
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error, mean_squared_error
from pandas.plotting import autocorrelation_plot

!pip install pmdarima
from pmdarima import auto_arima

from google.colab.patches import cv2_imshow

#### params

In [ ]:
mpl.rcParams['text.color'] = 'G'
plt.style.use('fivethirtyeight')

params = {'legend.fontsize':'x-large',
          'figure.figsize':(15, 8),
          'lines.linewidth':1.5,
          'axes.labelsize':'x-large',
          'axes.labelpad':15,
          'axes.labelweight':'bold',
          'axes.titlesize':35,
          'axes.titleweight':'bold',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'}
mpl.rcParams.update(params)

#### functions

In [ ]:
# Fonction affichant les 5 premières meilleurs combinaisaons de paramètres SARIMA
# En fonction de l'AIC et du BIC
def sarimax_gridsearch(ts, pdq, pdqs):
    '''
    Input: 
        ts : your time series data
        pdq : ARIMA combinations from above
        pdqs : seasonal ARIMA combinations from above
        maxiter : number of iterations, increase if your model isn't converging
        frequency : default='M' for month. Change to suit your time series frequency
            e.g. 'D' for day, 'H' for hour, 'Y' for year. 
        
    Return:
        Prints out top 5 parameter combinations
        Returns dataframe of parameter combinations ranked by BIC
    '''

    # Run a grid search with pdq and seasonal pdq parameters and get the best BIC value
    ans = []
    for comb in pdq:
        for combs in pdqs:
            try:
                mod = SARIMAX(ts,order=comb, seasonal_order=combs)
                output = mod.fit() 
                ans.append([comb, combs, output.bic, output.aic])
                #print('SARIMAX {} x {}12 : BIC Calculated ={} ; AIC Calculated : {}'.format(comb, combs, output.bic, output.aic))
            except:
                continue
            
    # Find the parameters with minimal BIC value

    # Convert into dataframe
    ans_df = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'bic', 'aic'])

    # Sort and return top 5 combinations
    ans_df = ans_df.sort_values(by=['bic', 'aic'],ascending=True)
    
    return ans_df.head()

# fonction affichant le Test de Dickey–Fuller avec les autocorrélogrammes ACF et PACF
def tsplot(y, lags=None, figsize=(20, 10), style='bmh'):
    
  #Determing rolling statistics
  window = 12
  rolmean = y.rolling(window=window).mean()
  rolstd = y.rolling(window=window).std()

  #Plot rolling statistics:
  fig = plt.figure(figsize=(15, 4))
  orig = plt.plot(y.iloc[window:], color='blue',label='Original')
  mean = plt.plot(rolmean, color='red', label='Rolling Mean')
  std = plt.plot(rolstd, color='black', label = 'Rolling Std')
  plt.legend(loc='best')
  plt.title('Rolling Mean & Standard Deviation')
  plt.show()

  """
  Test de Dickey–Fuller
  avec Autocorrélogrammes ACF et PACF

  """
  # perform augmented Dickey-Fuller test
  adf_test = adfuller(y, autolag='AIC')
  print(f"Results of Dickey-Fuller test :\n{'-'*50}")

  dftest = adfuller(y, autolag='AIC')
  dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','P-value','# Lags Used','Number of Observations Used'])
  print(dfoutput)
  print('\n')
  print("Is this data stationary ?")
  for key, value in dftest[4].items():
    print("\tCritical value {}: {} - The data is {} stationary with {}% confidence".format(key, value, "not" if value < dftest[0] else "", 100-int(key[:-1])))
    # dfoutput['Critical Value (%s)'%key] = value
  # print (dfoutput)
  # print("\n")

  alpha = 0.05
  p_value = adf_test[1]
  ADF_h0 = "CONCLUSION : time series is non-stationary, accept H0"
  ADF_ha = "CONCLUSION : time series is stationary, reject H0"

  if p_value < alpha:
    print("\n")
    print(ADF_ha)
  elif p_value > alpha:
    print("\n")
    print(ADF_h0)

  # visualisation des autocorrélogrammes ACF et PACF
  if not isinstance(y, pd.Series):
      y = pd.Series(y)

  with plt.style.context(style):
      fig = plt.figure(figsize=figsize)
      layout = (2, 2)
      ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
      acf_ax = plt.subplot2grid(layout, (1, 0))
      pacf_ax = plt.subplot2grid(layout, (1, 1))

      y.plot(ax=ts_ax)
      p_value = sm.tsa.stattools.adfuller(y)[1]
      ts_ax.set_title('Time Series Analysis Plots\n Dickey-Fuller: p={0:.5f}'.format(p_value))
      sm.tsa.graphics.plot_acf(y, lags=lags, ax=acf_ax)
      sm.tsa.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
      plt.tight_layout()

In [ ]:
# decomposition
# additive
def add_decomp(variable):
  cols = ["{}".format(col_name) for col_name in variable.columns]
  str = f"Additive decomposition for {cols}"

  add_decomp = seasonal_decompose(variable, model="additive")
  add_decomp.plot().suptitle(str, y=1.01, fontsize=15, fontweight="bold")

  add_decomp_results = pd.concat([add_decomp.seasonal, add_decomp.trend, add_decomp.resid, add_decomp.observed], axis=1)
  add_decomp_results.columns = ["seasonality", "trend", "residual", "actual_values"]
  print("Additive : ")
  display(add_decomp_results.head())

# multiplicative
def mul_decomp(variable):
  cols = ["{}".format(col_name) for col_name in variable.columns]
  str = f"Multiplicative decomposition for {cols}"

  mul_decomp = seasonal_decompose(variable, model="multiplicative")
  mul_decomp.plot().suptitle(str, y=1.01, fontsize=15, fontweight="bold")

  mul_decomp_results = pd.concat([mul_decomp.seasonal, mul_decomp.trend, mul_decomp.resid, mul_decomp.observed], axis=1)
  mul_decomp_results.columns = ["seasonality", "trend", "residual", "actual_values"]
  print(f"{'-'*55}\nMultiplicative :")
  display(mul_decomp_results.head())

# stationarity test
def sty_test(variable):

  # # determining rolling stats
  # ma_mean = variable.rolling(window=12, center=True).mean()
  # ma_std = variable.rolling(window=12, center=True).std()

  # # plot rolling stats
  # orig = variable.plot(label="original data")
  # mean = ma_mean.plot(label="rolling mean")
  # # std = ma_std.plot(label="rolling std")
  
  # plt.title("Smoothed time series on moving average")
  # plt.suptitle("window = 12", y=0.9, fontsize=15, fontweight="bold")
  # plt.tight_layout()
  # plt.legend()
  # plt.show()

  # perform augmented Dickey-Fuller test
  adf_test = adfuller(y, autolag='AIC')
  print(f"Results of Dickey-Fuller test :\n{'-'*50}")

  dftest = adfuller(y, autolag='AIC')
  dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','P-value','# Lags Used','Number of Observations Used'])
  print(dfoutput)
  print('\n')
  print("Is this data stationary ?")
  for key, value in dftest[4].items():
    print("\tCritical value {}: {} - The data is {} stationary with {}% confidence".format(key, value, "not" if value < dftest[0] else "", 100-int(key[:-1])))
    # dfoutput['Critical Value (%s)'%key] = value
  # print (dfoutput)
  # print("\n")

  alpha = 0.05
  p_value = adf_test[1]
  ADF_h0 = "CONCLUSION : time series is non-stationary, accept H0"
  ADF_ha = "CONCLUSION : time series is stationary, reject H0"

  if p_value < alpha:
    print("\n")
    print(ADF_ha)
  elif p_value > alpha:
    print("\n")
    print(ADF_h0)

# normality test
def nrm_test(variable):
  
  # Shapiro-Wilk test
  print("Results of Shapiro-Wilk test :")
  print("-"*30)
  sw_results = st.shapiro(variable)
  sw_output = pd.Series(sw_results[0:2],
                        index=["Test statistic",
                               "p_value"])
  print(sw_output, "\n")
  
  alpha = 0.05
  p_value = sw_output[1]
  SW_h0 = "CONCLUSION : Distirbution is normal, accept H0"
  SW_ha = "CONCLUSION : Distribution is non-normal, reject H0"
  
  if p_value > alpha:
    print(f"p_value {p_value:.3f} > alpha")
  elif p_value < alpha:
    print(f"p_value {p_value:.3f} < alpha")

  if p_value > alpha:
    # print("\n")
    print(SW_h0)
  elif p_value < alpha:
    # print("\n")
    print(SW_ha)
  
  print("\n")

  # Kolmogorov-Smirnov test
  print("Results of Kolmogorov-Smirnov test :")
  print("-"*36)
  ks_results = sm.stats.stattools.jarque_bera(variable)
  ks_output = pd.Series(ks_results[0:2],
                        index=["KS_stats",
                               "p_value"])
  print(ks_output, "\n")
  
  alpha = 0.05
  p_value = ks_output[1]
  KS_h0 = "CONCLUSION : Data is normally distributed, accept H0"
  KS_ha = "CONCLUSION : Data series is non-normal, reject H0"
  
  if p_value > alpha:
    print(f"p_value {p_value:.3f} > alpha")
  elif p_value < alpha:
    print(f"p_value {p_value:.3f} < alpha")

  if p_value > alpha:
    # print("\n")
    print(KS_h0)
  elif p_value < alpha:
    # print("\n")
    print(KS_ha)
  
  print("\n")

  # Jarque-Bera test
  print("Results of Jarque-Bera test :")
  print("-"*30)
  jb_results = sm.stats.stattools.jarque_bera(variable)
  jb_output = pd.Series(jb_results[0:4],
                        index=["JB_stats",
                               "p_value",
                               "Skewness",
                               "Kurtosis"])
  print(jb_output, "\n")
  
  alpha = 0.05
  p_value = jb_output[1]
  JB_h0 = "CONCLUSION : Normal distribution, accept H0"
  JB_ha = "CONCLUSION : Non-normal distribution, reject H0"
  
  if p_value > alpha:
    print(f"p_value {p_value:.3f} > alpha")
  elif p_value < alpha:
    print(f"p_value {p_value:.3f} < alpha")

  if p_value > alpha:
    # print("\n")
    print(JB_h0)
  elif p_value < alpha:
    # print("\n")
    print(JB_ha)
  
  sns.displot(variable, kde=True, bins=10, aspect=1.25, height=7)
  plt.tight_layout()
  plt.show()

# residuals normality test
def bp_test(variable):
  print("Results of Breusch-Pagan test:")
  print("-"*30)
  bp_results = sm.stats.diagnostic.het_breuschpagan(variable.resid, variable.model.exog)
  bp_output = pd.Series(bp_results[0:4],
                        index=["Lagrange multiplier statistic",
                               "p_value",
                               "F value",
                               "F p_value"])
  print(bp_output, "\n")
  
  alpha = 0.05
  p_value = bp_output[1]
  BP_h0 = "CONCLUSION : Homoscedasticity is present (the residuals are distributed with equal variance), accept H0"
  BP_ha = "CONCLUSION : Heteroscedasticity is present (the residuals are not distributed with equal variance), reject H0"
  
  if p_value > alpha:
    print(f"p_value {p_value:.3f} > alpha")
  elif p_value < alpha:
    print(f"p_value {p_value:.3f} < alpha")

  if p_value > alpha:
    # print("\n")
    print(BP_h0)
  elif p_value < alpha:
    # print("\n")
    print(BP_ha)

# Significance test
def significance(x,y):

  print("Results of Significance test :")

  slope, intercept, r_value, p_value, std_err = st.linregress(x,y)
    
  alpha = 0.05
  p_value = p_value
  sg_h0 = "CONCLUSION : Non significant (X has no effect on Y), accept H0"
  sg_ha = "CONCLUSION : Statistically significant (X has enough effect on Y), reject H0"
  
  if p_value > alpha:
    print(f"p_value {p_value:.3f} > alpha")
  elif p_value < alpha:
    print(f"p_value {p_value:.3f} < alpha")

  if p_value > alpha:
    print("-"*30)
    # print("\n")
    print(sg_h0)
  elif p_value < alpha:
    print("-"*30)
    # print("\n")
    print(sg_ha)

### Datasets

In [ ]:
import warnings
warnings.simplefilter("ignore")

data = pd.read_csv("/content/drive/MyDrive/OC/P9/conso.csv", index_col="Mois", parse_dates=True)
paris = pd.read_excel("/content/drive/MyDrive/OC/P9/DJU.xlsx", skiprows=11)
arima = cv2.imread('/content/drive/MyDrive/OC/P9/arima chart.jpeg')
arima = cv2.resize(arima, (700, 350))

### Data preparation

#### consommation

In [ ]:
data.index.names = ["date"]
data = data[data["Territoire"]=="France"][["Consommation totale"]]
data.rename(columns={"Consommation totale":"raw"}, inplace=True)

# display(data.dtypes,
#         data.head(2),
#         data.shape,
#         data.groupby([data.index.year])[["raw"]].count(),
#         data.index)

In [ ]:
data.head()

In [ ]:
data.plot()

In [ ]:
plt.figure(figsize=(15,16))

plt.subplot(2,1,1)
sns.boxplot(data.index.year, data.raw)
plt.title("Consommation d'éléctricité\nannées 2012 - 2020")

plt.subplot(2,1,2)
sns.boxplot(data.index.month, data.raw)
plt.title("Consommation d'éléctricité\nmenseulle entre 2012 et 2020")

plt.tight_layout()
plt.show()

In [ ]:
data.raw.resample("Y").agg(["mean", "median", "min", "max", "std"])

#### degrés jours unifiés

In [ ]:
paris.drop(columns="Total", inplace=True)
paris.rename(columns={"Unnamed: 0":"date",
                          "JAN":"01",
                          "FÉV":"02",
                          "MAR":"03",
                          "AVR":"04",
                          "MAI":"05",
                          "JUN":"06",
                          "JUI":"07",
                          "AOÛ":"08",
                          "SEP":"09",
                          "OCT":"10",
                          "NOV":"11",
                          "DÉC":"12"}, inplace=True)
paris.drop(0, inplace=True)
paris.set_index("date", inplace=True)

impute = {'month':[],'dju':[]}

for year in paris.index.values:
    for month in paris.columns:
        impute['month'].append(f"{year}-{month}")
        impute['dju'].append(paris.loc[year,month])
        
dju = pd.DataFrame(impute)
dju['date'] = pd.to_datetime(dju['month'])
dju.set_index("date", inplace=True)
dju.drop(columns="month", inplace=True)

# display(dju.dtypes,
#         dju.head(2),
#         dju.shape,
#         dju.groupby([dju.index.year])[["dju"]].count(),
#         dju.index)

In [ ]:
dju.head()

In [ ]:
dju.plot()

In [ ]:
dju.dju.resample("Y").agg(["mean", "median", "min", "max", "std"])

#### merge

In [ ]:
df = pd.merge(data, dju, on="date", how="inner")
df.head(3)

### Data overview

In [ ]:
df.describe().T

In [ ]:
plt.subplot(2,1,1)
df.resample("Y")["raw"].plot(title="Consommation d'électricité par an")

plt.subplot(2,1,2)
df.resample("Y")["dju"].plot(title="Degré Jour Unifié")

plt.tight_layout()
plt.show()

### Data correction

#### finding the coefficient term from Linear Regression using OLS Summary

On va pouvoir effectuer une corrélation entre les variables raw et DJU. Ainsi, on obtiendra un modèle de régression linéaire qui nous permettra de corriger la consommation d'électricité en fonction de la température enregistrée.

In [ ]:
# linear regression plot
x = np.array(df['dju'])
y = np.array(df['raw'])

sns.regplot(x=x,
            y=y,
            line_kws={"color":"k", "lw":2},
            scatter_kws={"color":"darkorange", "alpha":0.7, "s":20})
plt.title("Regression lineaire :\nconsommation d'électricité et DJU")
plt.show()

In [ ]:
display(df[["raw", "dju"]].corr())
print("\n")

reg = smf.ols('raw ~ dju', data=df).fit()
print(reg.summary(), "\n")

significance(x,y)

Notre régression linéaire montre un coefficient de détermination de 0.946%. On en conclut qu'il y a une forte corrélation entre la consommation totale et la température en DJU. En effet, DJU explique ~95% de la consommation.

De plus, l'observation de notre graphique montre que cette corrélation est positive (plus la consommation augmente plus la différence de température est élevé).

Aussi, la p-value (0.00) est inférieure à 0.05 donc notre corrélation est statistiquement significative.

- **R-squared value:** $R^2$ is the coefficient of determination that tells us that how much percentage variation independent variable can be explained by independent variable. Here, 94.6 % variation in consommation can be explained by DJU. The maximum possible value of $R^2$  can be 1, means the larger the $R^2$  value better the regression.

- **Coefficient term:** The coefficient term tells the change in "consommation" for a unit change in "DJU". Therefore, if DJU rises by 1 unit, then consommation rises by 49.0051. If you are familiar with derivatives then you can relate it as the rate of change of DJU with respect to consommation.

#### residuals normality test

In [ ]:
# QQ plot
st.probplot(reg.resid, plot= plt)
plt.title("QQ plot"); plt.show()

# homoscedasticity test
print("HOMOSCEDASTICITY TEST")
display(bp_test(reg))
print("\n")

# gaussian distribution test
print("GAUSSIAN DISTRIBUTION TEST")
display(nrm_test(reg.resid))

On peut identifier une distribution des résidus satisfaisante, alignée avec la distribution théorique d'une loi normale. Un test de normalité est nécessaire pour valider ou rejeter cette intuition. Or, la p_value supérieure au seuil ne permet pas de rejeter l'hypothèse nulle H0 de normalité des résidus, le test est donc validé: on accept H0 (la distribution est gaussienne).

#### corrected consumption

On crée une nouvelle colonne dans notre dataframe qui contient les valeurs de la consommation totale corrigée grâce au paramètre de la régression linéaire que l'on a extrait plus tôt.

In [ ]:
# correction des données avec la regression lineaire
coef = reg.params["dju"]

df["conso"] = df.raw - df.dju * coef
display(df.dtypes,
        df.head(3),
        df.shape)

## Principal data

### Data overview

In [ ]:
df.describe().T

Une fois la correction de la consommation calculée, on la représente graphiquement pour voir la différence entre consommation totale et consommation totale corrigée de la température.

In [ ]:
plt.subplot(3,1,1)
df.resample("Y")["conso"].plot(title="Consommation (corrigée)")

plt.subplot(3,1,2)
df.resample("Y")["raw"].plot(title="Consommation (non corrigée)")

plt.subplot(3,1,3)
df.resample("Y")["dju"].plot(title="Degré Jour Unifié")

plt.tight_layout()
plt.show()

In [ ]:
df[["raw", "conso"]].plot()
plt.title("avant et après correction\nde la consommation d'electricité"); plt.show()

df[["raw", "conso"]].plot(subplots=True); plt.show()

The dataset trend shows that it is a kind of additive time series, but this feature dataset has also a seasonality in it.

In [ ]:
plt.figure(figsize=(27,16))

plt.subplot(2,2,1)
sns.boxplot(df.index.year, df.conso)
plt.title("Consommation corrigée\nannées 2012 - 2020")

plt.subplot(2,2,2)
sns.boxplot(df.index.year, df.raw)
plt.title("Consommation non-corrigée\nannées 2012 - 2020")

plt.subplot(2,2,3)
sns.boxplot(df.index.month, df.conso)
plt.title("Consommation corrigée (mensuelle)\nannées 2012 - 2020")

plt.subplot(2,2,4)
sns.boxplot(df.index.month, df.raw)
plt.title("Consommation non-corrigée (mensuelle)\nannées 2012 - 2020")

plt.tight_layout()
plt.show()

En ayant corrigé notre série temporelle sur la consommation d'électricité, on s'assure que les prédictions faites dans la suite de notre étude ne prenne en compte que les variations de consommation électrique sans être influencés par les variations de température extérieures.

### Guassian distribution test

In [ ]:
nrm_test(df.conso)

### Decomposition

Une fois la correction effectuée, on peut s'intéresser aux différents composants de la série temporelle.

In [ ]:
decompose_data = seasonal_decompose(df.conso, model="additive")
decompose_data.plot();

Here in the above chart, we can see the decomposed structure of data and the structure of the components in the data set which were affecting it. Let’s make a graph for available seasonality.

### Seasonality

Let’s check for a couple of years seasonality to know the similarity more accurately. I am choosing the years 2012, 2015, 2017, and 2020 for the test.

In [ ]:
fig, ax = plt.subplots(4)

ax[0].plot(df.loc["2012", "conso"], label='2012')
ax[0].legend()

ax[1].plot(df.loc["2015", "conso"], label='2015')
ax[1].legend()

ax[2].plot(df.loc["2017", "conso"], label='2017')
ax[2].legend()

ax[3].plot(df.loc["2020", "conso"], label='2020')
ax[3].legend()

plt.tight_layout()
plt.show()

In [ ]:
seasonality = decompose_data.seasonal
seasonality.plot(c="red", title="seasonality"); plt.legend(); plt.show()

In the seasonality graph, we can see the seasonality structure for every year, which is cyclic and repeatedly providing the same value.

### Deseasonalisation

In [ ]:
# différence avec la saisonnalité pour obtenir une série temporelle hors impact saisonnier
deseasonalized = df.conso - seasonality
moving_average = df.conso.rolling(window=12, center=True).mean()

# visualisation de la consommation en électricité avant et après désaisonnalisation
plt.plot(df['conso'], label='Consommation corrigée')
plt.plot(deseasonalized, label='Consommation désaisonnalisée')
plt.plot(moving_average, label='moyenne mobile')

plt.suptitle('window = 12', fontsize=15, y=0.86)
plt.title('Avant et après désaisonnalisation')

plt.legend()
plt.show()

### Differencing

In [ ]:
tsplot(df.conso)

In [ ]:
diff = df.conso - df.conso.shift(1)
tsplot(diff.dropna())

In [ ]:
seasonal_diff = df.conso - df.conso.shift(12)
seasonal_diff = df.conso - df.conso.shift(1)
tsplot(seasonal_diff.dropna())

## Prévision de la consommation sur un an

La phase de prédition se fera en utilisant la méthode de Holt-Winters (lissage exponentiel) puis la méthode SARIMA.

#### Holt-Winters method

La méthode de Holt-Winters génère des valeurs lissées de façon exponentielle pour le niveau, la tendance et l'ajustement saisonnier de la série temporelle. Cette méthode convient pour les séries temporelles dont la saisonnalité et la tendance restent stables dans le temps.

La méthode ExponentialSmoothing de statsmodels est utilisée pour modéliser le lissage exponentiel d'Holt-Winters.

On commence par définir les paramètres de notre modèle puis on le représente graphiquement. Ici les paramètres de notre modèle contiennent des informations sur la saisonnalité qui est égale à 12 et une tendance et une saisonnalité additives.

In [ ]:
holt_winters = ExponentialSmoothing(np.asarray(df["conso"]), seasonal_periods=12, trend='add', seasonal='add').fit()
predict_hw = holt_winters.forecast(12)

plt.plot(df["conso"], label='Consommation corrigée')
plt.plot(pd.date_range(df.index[len(df)-1], periods=12, freq='M'), predict_hw, label='Prévision Holt-Winters')

plt.title("2021 Holt-Winters Forecasting")
plt.legend()
plt.show()

La méthode de Holt-Winters est la plus raisonnable des méthodes de lissage exponentiel. La prévision sur un an de la consommation, corrigée de l'effet température, tient compte de la saisonnalité.

#### Holt-Winters :  analyse a posteriori

Une fois les paramètres de notre modèle définis, on évalue notre modèle. Pour cela, on enlève les données des 12 derniers mois dont on dispose puis on compare les valeurs enregistrées avec les valeurs prédites de nottre modèle.

On tronque la série de l’année 2020, qu’on cherche ensuite à prévoir à partir de l’historique 2012-2019. Cette analyse permet d'avoir une idée de la qualité prédictive du modèle choisi.

In [ ]:
set(df.index.year)

In [ ]:
# specify the train/test split length
split = int(len(df)*.89)
train = df[:split]
test = df[split:]

# visualization of train/test splitting
train.conso.plot()
test.conso.plot(figsize=(20, 6), title= 'Train - Test splitting', fontsize=14, c='r');

In [ ]:
train_hw = ExponentialSmoothing(np.asarray(train["conso"]), seasonal_periods=12, trend='add', seasonal='add').fit()
train_pred = train_hw.forecast(12)

plt.plot(df["conso"], label="Consommation corrigée")
plt.plot(pd.date_range(train.index[len(train)-1], periods=12, freq='M'), train_pred, label="Prévision Holt-Winters")

plt.title("Holt-Winters Model Evaluation")
plt.legend()
plt.show()

#### Holt-Winters : model evaluation

Graphiquement, on remarque que la prédiction a réussi à capturer la saisonnalité mais qu'il a du mal à prédire la baisse importante du début de l'année 2020. Comme vu dans la définition de la méthode de Holt-Winters, ce modèle est bon pour prédire des événements lorsque la saisonnalité et la tendance sont stables. Or, selon la décomposition de notre série temporelle on remarque que la tendance est stable jusque 2019 mais qu'ensuite, elle s'effondre au début de l'année 2020.

Pour pouvoir conclure à l'évaluation de notre modèle, on va ensuite calculer différents indices.

In [ ]:
y_pred = train_pred
actual_values = np.asarray(train['conso'].iloc[-12:])

mae = mean_absolute_error(actual_values, train_pred)
mse = mean_squared_error(actual_values, train_pred)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((actual_values - train_pred) / actual_values)) * 100

print('MAE = ', mae)
print('MSE = ', mse)
print('RMSE = ', rmse)
print('MAPE = ', mape)

Notre modèle obtient des scores plutôt bons avec une marge d'erreur (MAPE) de 1.75%. Cependant ce type de modélisation n'est pas adapté pour notre série temporelle où la tendance est à la baisse au début de l'année 2020.

#### Setting up SARIMA

Pour contrer le problème de la difficulté de traitement de la présence d'une tendance et/ou d'une saisonnalité changeante, on va pouvoir utiliser un modèle issu de la méthode SARIMA. Cette méthode combine une méthode auto-regressive à une méthode de moyenne mobile.

Plusieurs étapes sont nécessaires :

- Identifier/confirmer la saisonnalités (autocorrélogrammes)
- Stationnariser la série temporelle (différenciation)
- Déterminer des ordres optimaux plausibles
- Estimer les paramètres et les départager par l’AIC (ou le BIC)
- Valider ou non le modèle par un diagnostique des résidus (test, représentation graphique, autocorrélogramme)
- Confirmer le(s) choix en simulant la prévision.



##### Stationarity

Une série présentant une tendance et/ou une saisonnalité ne pourra pas être modélisée par un processus stationnaire. Une technique communément employée est de travailler non pas sur la série mais sur des différences de la série.

On va devoir vérifier la non-stationnarité de notre série temporelle avant de pouvoir réaliser la prévision par SARIMA. On commence donc par créer une variable qui ne contient que notre série temporelle de la consommation corrigée.

In [ ]:
y = df["conso"]

On va ensuite s'intéresser à la saisonnalité et au calcul de la stationnarité de notre série temporelle.

In [ ]:
tsplot(y, lags=55)

La sortie ACF présente une décroissance lente vers 0, ce qui traduit un problème de non-stationnarité. Confirmation également par le test de Dickey-Fuller portant pour hypothèse nulle la non-stationnarité de la série, ne pouvant pas être rejetée. On effectue donc une première différenciation.

In [ ]:
diff = df.conso - df.conso.shift(1)
tsplot(diff.dropna(), lags=60)

La sortie ACF de la série ainsi différenciée présente encore une décroissance lente vers 0 pour les multiples de 12. On effectue cette fois la différenciation d'ordre 12.

In [ ]:
diff_2 = diff - diff.shift(12)
tsplot(diff_2.dropna(), lags=60)

Cette fois-ci les résultats sont satisfaisants, nous pouvons nous appuyer sur les autocorrélogrammes simple et partiels estimés.

##### Configuring parameters

In [ ]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
# print('Examples of parameter for SARIMA...')
# print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
# print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
# print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
# print('SARIMAX: {} x {}\n'.format(pdq[2], seasonal_pdq[4]))

run_res = []

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
            results = mod.fit()
            run_res.append({'order':param,
                            'seasonal': param_seasonal,
                            'aic': results.aic,
                            'mse': mse,
                            'rmse': sqrt(mse)
                           })
        except:
            continue

run_res_df = pd.DataFrame(run_res)
run_res_df

##### Evaluating the model

In [ ]:
min = run_res_df.aic.min()
run_res_df[run_res_df["aic"] == min]

In [ ]:
max = run_res_df.aic.max()
run_res_df[run_res_df["aic"] == max]

In [ ]:
run_res_df.duplicated().sum()

According Peterson, T. (2014) the AIC (Akaike information criterion) is an estimator of the relative quality of statistical models for a given set of data. Given a collection of models for the data, AIC estimates the quality of each model, relative to each of the other models. The low AIC value the better. Our output suggests that SARIMAX(0, 1, 1)	(0, 1, 1, 12) with AIC value of 1367.02 is the best combination, so we should consider this to be optimal option.

In the “mod = sm.tsa.statespace.SARIMAX” command we need to set up the chosen combination.

In [ ]:
mod = sm.tsa.statespace.SARIMAX(y,
                                order=(0, 1, 1),
                                seasonal_order=(0, 1, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results = mod.fit()
# print(results.summary().tables[1])
print("\n", results.summary())

Selon le test de significativité des paramètres obtenu dans le modèle, on constate que les coefficients/paramètres MA simple et MA saisonnière sont tous significativement différents de 0, à un niveau de test de 5%. En effet, leur p_value est inférieure à 5% niveau de test.

##### White noise

À l'issue de la modélisation, on va espérer sue le résidu obtenu soit un bruit blanc. C'est-à-dire, qu'on ait pu extraire au moins toutes les dépendances linéaires temporelles au sein de la série temporelle initale.  

Si nous avions voulu faire un test de blancheur du résidu de ce modèle, Nous pouvions utiliser la méthode de Ljung-Box.

In [ ]:
print('Retard : p-value')
for elt in [6, 12, 18, 24, 30, 36]:
    print('{} : {}'.format(elt, acorr_ljungbox(results.resid, lags=elt)[1].mean()))

On obtient ici les p_values pour les ordres 6 à 36, et pour rappel, le test effectué est un test de Ljung-Box. On constate là, que les p_values ne sont pas inférieures au nivea de test de 5%, donc on ne rejette pas l'hypothèse nulle (H0: le résidu suit un bruit blanc). Donc c'est plutôt salvateur.

Ce modèle là est du coup satisfaisant pour les résidus, mais également pour les paramètres.

> **RECAP :** les deux coefficients de nos paramètres $\theta1$ et $\theta1_{ saisonnier}$ sont très significatifs. Et pour que le modèle soit correcte, on a vérifié que le test de blancheur ne soit pas rejeté, et selon le test de Ljung-Box, effectivement le test de blancheur n'est pas rejeté, donc les résidus suivent un bruit blanche.

##### Residual normality tests

Dans le cadre de prévisions, il convient de vérifier la normalité des résidus pour tester l'adéquation de nos modèles. La normalité peut-être détectée de façon graphique, mais des tests statistiques amènent un point de vue objectif non négligeable.

In [ ]:
results.plot_diagnostics(figsize=(20,8))
plt.grid()
plt.tight_layout()
plt.show()

With the diagnostic above we can visualize important information as the distribution and the Auto correlation function ACF (correlogram). Values upward the “0” has some correlation over the time series data. Values near to “1” demonstrates strongest correlation.

La représentation "Standardized residual" et "Correlogram" confirment qu'il n'y a pas de corrélation des résidus. Les résidus sont normalement distribués KDE vs distribution normale - N (0,1). La distribution ordonnée des résidus représentée par le du Q-Q plot est globalement satisfaisant, il y a quand même des petites divergences vers les queues de distribution. Il est intéressant de coupler l'approche visuelle par des tests statistiques.

In [ ]:
nrm_test(results.resid)

Ce test ne prend pas en compte les premiers résidus d'après la documentation, ce qui rappelle le premier constat du Q-Q plot, les premiers résidus divergent légèrement d'un schéma gaussien, le test ne les prend pas en compte est devient donc concluant sans pouvoir rejeter l'hypothèse nulle de normalité des résidus dans ce cadre précis.

Ici, l'hypothèse de normalité est remise en cause par Shapiro (p-value < 0.05).

Néanmoins, l'observation des résidus, le fait qu'ils ne soient pas très différents d'une distribution symétrique, et le fait que l'échantillon soit de taille suffisante permettent de dire que les résultats obtenus par le modèle ne sont pas absurdes, même si le résidu n'est pas considéré comme étant gaussien.

#### SARIMA method

Les modèles SARIMA permettent de modéliser des séries qui présentent une saisonnalité.

Estimer un modèle SARIMA se ramène en pratique à l'estimation d'un modèle ARMA sur la série différenciée.

In [ ]:
model = results.get_forecast(12)
pred = model.predicted_mean
pred_l = model.conf_int(alpha=0.03)
pred_u = model.conf_int(alpha=0.03)

plt.plot(y, label='consommation')
plt.plot(pd.date_range(y.index[len(y)-1], periods=12, freq='M'), pred, color='red', label='prévision')
plt.plot(pd.date_range(y.index[len(df)-1], periods=12, freq='M'), pred_l, color='gray', linestyle=':')
plt.plot(pd.date_range(y.index[len(df)-1], periods=12, freq='M'), pred_u, color='gray', linestyle=':')

plt.legend()
plt.show()

#### SARIMA :  analyse a posteriori

L'analyse a posteriori permet de quantifier les écarts entre les prévisions et les réalisations, en tronquant la série d'un certain nombre de points (notons que le modèle doit être correctement estimé sur la série tronquée.

In [ ]:
# pred = results.get_prediction(start=pd.to_datetime('2019-01-01'), dynamic=False)
# pred_ci = pred.conf_int()
# ax = y['2012':].plot(label='observed')
# pred.predicted_mean.plot(ax=ax, label='one-step ahead Forecast', alpha=.7, figsize=(20, 8))
# ax.fill_between(pred_ci.index,
#                 pred_ci.iloc[:, 0],
#                 pred_ci.iloc[:, 1], color='k', alpha=.2)
# ax.set_xlabel('Date')
# ax.set_ylabel('consommation')
# plt.legend()
# plt.show()

In [ ]:
pred_actual_values = y[-12:]
past_values = y.shift(-12).dropna()

model = SARIMAX(past_values, order=(0,1,1), seasonal_order=(0,1,1,12))
model_results = model.fit()
print("\n", model_results.summary())

print('\n', 'Retard : p-value')
for elt in [6, 12, 18, 24, 30, 36]:
    print('{} : {}'.format(elt, acorr_ljungbox(model_results.resid, lags=elt)[1].mean()))

Les tests de significativité des paramètres et de blancheur du résidu sont validés au niveau 5%.

In [ ]:
pred_trunc_model = model_results.get_forecast(12)
pred_trunc = pred_trunc_model.predicted_mean
pred_l_trunc = pred_trunc_model.conf_int(alpha=0.05)
pred_u_trunc = pred_trunc_model.conf_int(alpha=0.05)

plt.plot(past_values, label='consommation')
plt.plot(pred_actual_values, label = 'consommation actuelle', color='red', lw=3, alpha=0.45)
plt.plot(pd.date_range(past_values.index[len(past_values)-1], periods=12, freq='M'), pred_trunc, color='k', label='consmmation prédit', linestyle='--')
plt.plot(pd.date_range(past_values.index[len(past_values)-1], periods=12, freq='M'), pred_l_trunc, color='g', linestyle=':', lw=0.75)
plt.plot(pd.date_range(past_values.index[len(past_values)-1], periods=12, freq='M'), pred_u_trunc, color='g', linestyle=':', lw=0.75)

plt.legend()
plt.show()

Zoom sur l'année 2019 à 2020

In [ ]:
pred_trunc_model = model_results.get_forecast(12)
pred_trunc = pred_trunc_model.predicted_mean
pred_l_trunc = pred_trunc_model.conf_int(alpha=0.05)
pred_u_trunc = pred_trunc_model.conf_int(alpha=0.05)


plt.plot(pred_actual_values, label = 'consommation actuelle', color='red', lw=3, alpha=0.45)
plt.plot(pd.date_range(past_values.index[len(past_values)-1], periods=12, freq='M'), pred_trunc, color='k', label='consmmation prédit', linestyle='--')
plt.plot(pd.date_range(past_values.index[len(past_values)-1], periods=12, freq='M'), pred_l_trunc, color='g', linestyle=':', lw=0.75)
plt.plot(pd.date_range(past_values.index[len(past_values)-1], periods=12, freq='M'), pred_u_trunc, color='g', linestyle=':', lw=0.75)

plt.legend()
plt.show()

#### SARIMA : model evaluation

On utilise des critères d'erreur comme
- l'erreur quadratique moyenne (RMSE = Root Mean Square Error)
- ou
- l'erreur relative absolue moyenne (MAPE = Mean Average Percentage Error)

In [ ]:
mae = mean_absolute_error(pred_actual_values, pred_trunc)
mse = mean_squared_error(pred_actual_values, pred_trunc)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((pred_actual_values - pred_trunc) / pred_trunc)) * 100

print('MAE = ', mae)
print('MSE = ', mse)
print('RMSE = ', rmse)
print('MAPE = ', mape)

L’interprétation des critères d’erreur dépend de la série et de la qualité de prévision exigée. Dans le cas présent, un MAPE de 0.9% semble satisfaisant a priori.

Prévision à 12 mois performante, la méthode SARIMA semble mieux prendre en compte les impacts saisonniers (pics et creux) et se montre plus performante que la méthode Holt-Winters où le MAPE était égale à 1.75%.

### Conclusion

Dans cette étude, nous avons utilisé les données mensuelles de consommation d’électricité en France, ainsi que l'effet de température lié au chauffage pour tester des modèles de prévison de cette consommation à 12 mois. L’exploration de ces données nous a révélé des caracteristiques tels que la non stationnarité de la série de consommation d’électricité et sa très forte inertie (autocorrélation forte et longue).

Les prévisions par la méthode de lissage exponentiel Holt-Winters et SARIMA sont globablement satisfaisantes, avec l’erreur absolue moyenne en pourcentage au dessous de 2%. Par ailleurs, le modèle SARIMA sera retenu prioritairement pour prédire la consommation d’électricité plus ponctuelle (à court terme), il sera plus performant et plus robuste dans des prévisions devant prendre en compte de forts impacts saisonniers.

# Sources

1. [Skewness and Kurtosis in finance](https://www.youtube.com/watch?v=WjdpbO3s-sQ&list=TLPQMzEwMzIwMjK1aBgSdlrfHw&index=2&ab_channel=DataCamp)
2. [ARMA : understanding ACF and PACF charts](https://medium.com/@ooemma83/how-to-interpret-acf-and-pacf-plots-for-identifying-ar-ma-arma-or-arima-models-498717e815b6)
3. [Matplotlib styling](https://matplotlib.org/2.1.2/api/_as_gen/matplotlib.pyplot.plot.html)
04. [OLS summary](https://www.geeksforgeeks.org/interpreting-the-results-of-linear-regression-using-ols-summary/)
05. [Simple linear regression excercice](https://www.youtube.com/watch?v=8jazNUpO3lQ&ab_channel=codebasics)
6. [Normality tests](https://towardsdatascience.com/6-ways-to-test-for-a-normal-distribution-which-one-to-use-9dcf47d8fa93)
7. [Homoscedasticity normality test for residuals](https://www.statology.org/breusch-pagan-test/)
8. [Perform Breusch-Pagan test](https://www.statology.org/breusch-pagan-test-python/)
9. [Residual plot](https://www.statology.org/residual-plot-python/)
10. [Customizing Matplotlib with style sheets and rcParams](https://matplotlib.org/stable/tutorials/introductory/customizing.html)
11. [Writing mathematical expressions in Matplotlib](https://matplotlib.org/3.5.0/tutorials/text/mathtext.html)